In [1]:
import csv,json
import networkx as nx
import collections
import json
import numpy as np

In [3]:
GH=nx.Graph()
#GH is the Graph after applying Min-Max Weight upto three decimal places
with open('MR_HsPth_Norm.csv') as csvfile:
    fr = csv.reader(csvfile, delimiter=',')
    for i,e in enumerate(fr):
        GH.add_edge(e[0],e[1],weight=e[2])

In [4]:
#Human-nCoV interaction Network
pds=[];GnC=nx.Graph();c=0
f=open("SARS-CoV2_Protein.txt","r")
for ln in f:
    pd=ln.split('\n')[0]
    pds.append(pd)
print(type(pds))
for e in GH.edges():
    if (((e[0] in pds) and (e[1] not in pds)) or ((e[1] in pds )and (e[0] not in pds))):
        GnC.add_edge(e[0],e[1],weight=GH[e[0]][e[1]]['weight'])
print(len(GnC.nodes()))
print(len(GnC.edges()))

<type 'list'>
19173
206516


In [36]:
GNn=nx.Graph()# GNn is the graph with 0.15 threshold

for e in GnC.edges():
    if float(GnC[e[0]][e[1]]['weight'])>=0.001:
        GNn.add_edge(e[0],e[1],weight=GnC[e[0]][e[1]]['weight'])
print(len(GNn.nodes()))
print(len(GNn.edges()))

18936
166382


In [23]:
Pid_H=[]#Human Protein List
f=open("Human_proteins.txt","r")
for ln in f:
    pd=ln.split("\n")[0].split("\t")[0]
    Pid_H.append(pd)
print(len(Pid_H))

19179


In [37]:
PID_Hm=[]
PID_nC=[]
PID_CV=[]
for n in GNn.nodes():
    if n in Pid_H:
        PID_Hm.append(n)
    elif n in pds:
        PID_nC.append(n)
    else:
        PID_CV.append(n)
print("Human Protein Involved=>",len(PID_Hm))
print("nCoV Protein Involved=>",len(PID_nC))
print("Other Corona Virus Protein Involved=>",len(PID_CV))

('Human Protein Involved=>', 18816)
('nCoV Protein Involved=>', 14)
('Other Corona Virus Protein Involved=>', 106)


In [38]:
GHq=nx.Graph();G2=nx.Graph()
with open('Gordon_HighQuality.csv') as csvfile:
    fr = csv.reader(csvfile, delimiter=',')
    for i,e in enumerate(fr):
        if i>0:
            GHq.add_edge(e[0],e[1])
print(len(GHq.edges()))
print(len(GHq.nodes()))
for row in GHq.edges():
    if (GNn.has_edge(row[0],row[1])) or (GNn.has_edge(row[1],row[0])):
            G2.add_edge(row[0],row[1])
print("Number of Edges==>",len(G2.edges()))
print("Number of Nodes==>",len(G2.nodes()))  
#print(G2.nodes())

332
359
('Number of Edges==>', 99)
('Number of Nodes==>', 109)


In [26]:
PID_Gr_H=[];PID_Gr_C=[]
with open('Gordon_HighQuality.csv') as csvfile:
    fr = csv.reader(csvfile, delimiter=',')
    for i,e in enumerate(fr):
        if i>0:
            PID_Gr_H.append(e[1])
            PID_Gr_C.append(e[0])
PID_Gr_H=list(set(PID_Gr_H))
PID_Gr_C=list(set(PID_Gr_C))
print(len(PID_Gr_H))
print(len(PID_Gr_C))

332
27


In [39]:
PID_Fz_H=[];PID_Fz_C=[]
for row in G2.nodes():
    if (row in PID_Hm):# or (row in PID_Hm):
        PID_Fz_H.append(row)
    if (row in PID_nC):#or (row in PID_nC):
        PID_Fz_C.append(row)
PID_Fz_H=list(set(PID_Fz_H))
PID_Fz_C=list(set(PID_Fz_C))
print("Number of Human Nodes==>",len(PID_Fz_H))
print("Number of nCoV Nodes==>",len(PID_Fz_C))
#print(PID_Fz_C)
#print(PID_Fz_H)

('Number of Human Nodes==>', 99)
('Number of nCoV Nodes==>', 10)


In [47]:
G3=nx.Graph()
for e in G2.edges():
    if (e[0] in PID_Fz_H and e[1] in PID_Fz_C) or (e[0] in PID_Fz_C and e[1] in PID_Fz_H):
        G3.add_edge(e[0],e[1])
print(len(G3.edges()))
print(len(G3.nodes()))

99
109


In [43]:
f=open("GordonHQ_0.001.csv","w")
s=''
for e in GNn.edges():
    if G2.has_edge(e[0],e[1]):
        s+="{},{},{:3f}\n".format(e[0],e[1],(float(GNn[e[0]][e[1]]['weight'])))
f.write(s);f.close()